[View in Colaboratory](https://colab.research.google.com/github/pmanlukas/colabSVM/blob/master/linear_classifier.ipynb)

# Linear Classifier
In this notebook a linear classifier based on a SVM is developed that is used as benchmark for the neural network based classifier

In [1]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18298 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .../0


Creating config file /etc/apt/apt.conf.d/50unattended-upgrades with new version
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up dirmngr (2.1.15-1ubuntu8) ...
Setting up cron (3.0pl1-128ubuntu5) ...
Adding group `crontab' (GID 102) ...
Done.
update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
update-rc.d: warning: stop runlevel arguments (1) do not match cron Default-Stop values (none)
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Setting up kmod (24-1ubuntu2) ...
Setting up libdbus-glib-1-2:amd64 (0.108-2) ...
Setting up python3-gi (3.24.1-2build1) ...
Setting up module-init-tools (24-1ubuntu2) ...
Setting up python3-software-properties (0.96.24.17) ...
Setting up dbus (1.10.22-1ubuntu1) ...
Setting up python-apt (1.4.0~beta3build2) ...
Setting up python3-dbus (1.2.4-1buil

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [3]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

In [5]:
!ls /content/drive/openapi-data/

glove.6B.zip		 path_labels.csv	   structure_labels.csv
mnist.py		 pickle			   weights.best.hdf5
new_path_labels0518.csv  PoC Classifier Path.odt   wordvector
path_labels0518.csv	 structure_labels0518.csv


In [0]:
import os
os.chdir("drive/openapi-data")

In [7]:
!ls

glove.6B.zip		 path_labels.csv	   structure_labels.csv
mnist.py		 pickle			   weights.best.hdf5
new_path_labels0518.csv  PoC Classifier Path.odt   wordvector
path_labels0518.csv	 structure_labels0518.csv


In [0]:
import pickle

In [0]:
training_data = dict()

with open('pickle/specjson210518.pkl', 'rb') as handle:
    training_data = pickle.load(handle)

In [0]:
keys = list(training_data.keys())

In [11]:
training_data[keys[0]]

{'basePath': '/forex-quotes',
 'host': '1forge.com',
 'info': {'contact': {'email': 'contact@1forge.com',
   'name': '1Forge',
   'url': 'http://1forge.com'},
  'description': 'Stock and Forex Data and Realtime Quotes',
  'title': '1Forge Finance APIs',
  'version': '0.0.1',
  'x-apisguru-categories': ['financial'],
  'x-logo': {'backgroundColor': '#24292e',
   'url': 'https://api.apis.guru/v2/cache/logo/http_1forge.com_logo.png'},
  'x-origin': [{'format': 'swagger',
    'url': 'http://1forge.com/openapi.json',
    'version': '2.0'}],
  'x-preferred': True,
  'x-providerName': '1forge.com'},
 'paths': {'/quotes': {'get': {'description': 'Get quotes',
    'externalDocs': {'description': 'Find out more',
     'url': 'http://1forge.com/forex-data-api'},
    'responses': {'200': {'description': 'A list of quotes'}},
    'summary': 'Get quotes for all symbols',
    'tags': ['forex', 'finance', 'quotes']}},
  '/symbols': {'get': {'description': 'Symbol List',
    'externalDocs': {'descripti

In [0]:
from collections import Counter
from datetime import datetime
 
import json
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [0]:
labelsP = pd.read_csv("path_labels0518.csv").values
labelsS  = pd.read_csv("structure_labels0518.csv").values

In [14]:
texts = []   # list of text samples
for spec in training_data:
  strJson = json.dumps(training_data[spec])
  texts.append(strJson)

print(str(len(texts)))

1034


In [15]:
labelsPath = list()
labelsStruct = list()


for labs in labelsS:
  labelsStruct.append(labs[1])

for labp in labelsP:
  labelsPath.append(labp[1])
  
print(str(len(labelsPath)))
print(str(len(labelsStruct)))






1034
1034


In [0]:
from sklearn.model_selection import train_test_split

#split the data and labels into a test and train set
X_trainS, X_testS, y_trainS, y_testS = train_test_split(texts, labelsStruct, test_size=0.3, random_state=0)




In [0]:
#split the data and labels into a test and train set
X_trainP, X_testP, y_trainP, y_testP = train_test_split(texts, labelsPath, test_size=0.3, random_state=0)



In [0]:
#split for category texts and labels
X_trainC, X_testC, y_trainC, y_testC = train_test_split(texts, labelsPath, test_size=0.3, random_state=0)

In [0]:
from sklearn import model_selection, preprocessing, linear_model

#encoder for labels of category
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

## model architecture

the first model is for structure labels

In [0]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
 

In [0]:
#vectorizer based on TF-IDF
vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')

tf_transformer = vectorizer.fit(texts)

In [0]:
#Classifiers, Naive Bayes and linear SVM based
clfSVC = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter=15, random_state=42)
clfNB = MultinomialNB()

In [24]:
#TODO: change code to use sklearn Pipeline
  
t1 = datetime.now()


#fit to vectorizer
XtrainS = tf_transformer.transform(X_trainS)
XtestS = tf_transformer.transform(X_testS)


print(datetime.now() - t1)
print(XtrainS.shape)

#fit model to data
model_svm = clfSVC.fit(XtrainS, y_trainS)
model_nb = clfNB.fit(XtrainS, y_trainS)

#cross validation
score_svm = cross_val_score(model_svm, XtrainS, y_trainS, cv=10, n_jobs=-1)
score_nb = cross_val_score(model_nb, XtrainS, y_trainS, cv=10, n_jobs=-1)
 
print(datetime.now() - t1)
print("CV of SVM:")
print(score_svm)
print(sum(score_svm) / len(score_svm))
print("CV of nb:")
print(score_nb)
print(sum(score_nb) / len(score_nb))

0:00:14.544077
(723, 49606)
0:00:18.515373
CV of SVM:
[0.81818182 0.74666667 0.82432432 0.79452055 0.73611111 0.75
 0.78873239 0.85714286 0.84285714 0.73913043]
0.789766729737793
CV of nb:
[0.64935065 0.68       0.60810811 0.57534247 0.61111111 0.65277778
 0.61971831 0.65714286 0.67142857 0.60869565]
0.6333675502705568


In [26]:
# TODO: add test based on trained classifier
predictions_svm = cross_val_predict(model_svm, XtestS, y_testS, cv=10)
print(predictions_svm)


predictions_nb = cross_val_predict(model_nb, XtestS, y_testS, cv=10)
print(predictions_nb)

[4 7 4 7 7 8 7 7 7 8 5 4 7 3 2 8 6 7 3 3 5 7 3 4 4 4 5 3 6 3 1 5 3 5 6 7 5
 5 7 3 7 4 3 5 6 8 1 1 5 6 6 7 3 3 4 3 2 3 7 4 7 5 5 7 1 7 6 4 3 5 1 4 8 1
 1 7 2 3 6 8 5 7 5 4 1 3 6 4 7 8 8 7 3 4 2 8 5 3 8 7 7 2 8 3 8 7 7 3 4 1 3
 7 3 7 3 3 7 4 1 2 7 3 8 7 8 4 7 7 4 6 1 3 7 1 2 2 5 4 1 4 6 3 1 4 1 4 5 7
 3 3 1 6 6 8 4 4 8 1 4 8 1 2 4 5 7 7 3 6 4 6 6 1 7 5 5 7 5 8 2 7 3 7 3 6 2
 1 1 4 2 1 4 1 3 2 8 3 4 1 3 3 5 6 8 7 2 8 1 3 3 5 3 8 7 7 8 3 7 6 7 7 7 8
 6 1 6 4 3 8 8 6 5 3 7 7 8 6 7 1 7 7 1 3 7 4 3 5 4 5 3 8 4 1 4 1 5 7 6 8 3
 5 7 7 7 3 4 1 7 7 5 5 4 3 4 8 3 3 1 7 7 7 6 1 3 1 5 4 8 6 6 5 3 4 1 4 1 7
 7 3 5 4 7 8 7 1 1 7 4 7 7 7 7]
[3 7 7 7 7 8 7 7 7 8 5 7 7 3 7 8 7 7 7 3 5 7 3 7 7 4 5 7 6 7 3 5 3 5 7 7 5
 5 7 3 7 3 3 5 7 8 7 3 5 7 7 7 7 3 7 7 8 3 7 7 7 5 5 7 3 7 6 4 3 5 7 7 8 7
 7 7 8 3 7 8 5 7 5 7 3 3 7 7 7 8 8 7 3 8 8 8 5 7 8 7 7 7 8 7 8 7 7 3 3 7 3
 7 3 7 3 7 7 7 3 7 7 7 8 7 8 3 7 7 7 7 7 3 7 3 8 8 5 7 7 7 7 3 7 4 7 3 5 7
 3 3 7 7 7 8 7 3 8 3 7 8 7 8 3 5 7 7 7 7 3 7 7 7 7 5 5 7 5 8 7 7 7 7

The following model is created for the paths label

In [27]:
#TODO: change code to use sklearn Pipeline
  
t1 = datetime.now()



XtrainP = tf_transformer.transform(X_trainP)
XtestP = tf_transformer.transform(X_testP)


print(datetime.now() - t1)
print(XtrainP.shape)

# TODO: adjust classifier to generate better accuracy
model_svm_P = clfSVC.fit(XtrainS, y_trainS)
model_nb_P = clfNB.fit(XtrainS, y_trainS)

#cross validation
score_svm_P = cross_val_score(model_svm_P, XtrainP, y_trainP, cv=10, n_jobs=-1)
score_nb_P = cross_val_score(model_nb_P, XtrainP, y_trainP, cv=10, n_jobs=-1)
 
print(datetime.now() - t1)
print("CV of SVM:")
print(score_svm_P)
print(sum(score_svm_P) / len(score_svm_P))
print("CV of nb:")
print(score_nb_P)
print(sum(score_nb_P) / len(score_nb_P))

0:00:14.258098
(723, 49606)


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


0:00:20.698799
CV of SVM:
[0.25316456 0.25974026 0.28       0.28       0.22666667 0.18055556
 0.28169014 0.34782609 0.26865672 0.25396825]
0.2632268237112264
CV of nb:
[0.20253165 0.18181818 0.21333333 0.17333333 0.28       0.16666667
 0.28169014 0.2173913  0.20895522 0.15873016]
0.2084449988524788


In [0]:
#TODO: add prediction for Path
predictions_svm_P = cross_val_predict(model_svm_P, XtestP, y_testP, cv=4)
plt.scatter(y_testP, predictions_svm_P)



## package the model

In [0]:
from sklearn.externals import joblib

In [0]:
# TODO: search for more efficient way to deserialize the vectorizer

joblib.dump(vectorizer, "pickle/tfidf_vectorizer.pickle")

In [30]:
joblib.dump(model_svm, "pickle/model_svm.pickle")


['pickle/model_svm.pickle']

In [31]:
joblib.dump(model_svm_P, "pickle/model_svm_P.pickle")

['pickle/model_svm_P.pickle']